In [21]:
!pip install tweet-preprocessor

  Running setup.py bdist_wheel for tweet-preprocessor ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/64/d9/5c/c72815d52f5770f229b909840516fd7a36e11bd57d4b8f28dd
Successfully built tweet-preprocessor


In [7]:
cd

cleaning_eg_stream.ipynb  twitter_stream_eg_notebook.ipynb


In [1]:
import preprocessor as p

In [2]:
import pandas as pd
import json

In [19]:
cd eg_twitter_raw


/home/jovyan/capstone/eg_twitter_raw


### Turn query documents (tweets) into generators

In [20]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [21]:
query_gen = load_tweets('stream_ازيك.jsonl')

### Turn query generators into dataframes

In [22]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [23]:
query_df = tweet_to_df(query_gen)

In [24]:
query_df.sample(4)

,geo,screen_name,text
622,Mansoura,nour_elkassab,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط
276,"القليوبية, مصر",Mohamed58531167,@shamsamr6291 علي فكره دي ميزه من مميزات الشعب...
260,None,fawaz2050,ازيك يا طويطرية https://t.co/kCYycumFKg
427,elmansoura,DohaAbdelbadea,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط


### Clean tweets (remove emojis, links, http, etc.)

In [25]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(tweet)) 

query_df['cleaned_text']=cleaned_text

In [26]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [27]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [28]:

query_df.sample(5)

,geo,screen_name,text,cleaned_text,cleaned_name,cleaned_geo
369,somewhere,massa_ayman,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط,: انا في موود لو حد قالي ازيك هعيط,massa_ayman,somewhere
474,None,donyaFmohamed,RT @Sam_ar_: انا في موود لو حد قالي ازيك هعيط,: انا في موود لو حد قالي ازيك هعيط,donyaFmohamed,None
232,None,gmJEO4Sc4zRbh9I,@22mosalah @RamyCol ازيك كابتن محمد اسمي ابراه...,ازيك كابتن محمد اسمي ابراهيم مولود في قريه صغي...,gmJEO4Sc4zRbh9I,None
243,جيزان،الممكة العربية السعودية,Alkhormi6,@sooii70 مساالخير\nازيك يادلوعه برسلك الى تبين...,مساالخير ازيك يادلوعه برسلك الى تبين ولاابي من...,Alkhormi6,جيزان،الممكة العربية السعودية
549,غيمة إيجار ☁️,emanweel11,ازيك وازي امك 😂 https://t.co/PeyOVtbNhH,ازيك وازي امك,emanweel11,غيمة إيجار ️


### Remove duplicated tweets

In [29]:
query_df['cleaned_text'].duplicated().sample(5)

131    False
217    False
232    False
320    False
552     True
Name: cleaned_text, dtype: bool

In [30]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep='last')
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [31]:
query_cleaned.shape

(330, 3)

In [32]:
query_cleaned.sample(4)

,cleaned_text,cleaned_name,cleaned_geo
75,ازيك يا ست شهد اخبارك ايه,shikoo_911,Egypt
522,ازيك ي هسااان — هسااان.!! الحمد لله..مين؟,hassanbelal1230,None
302,هلابك ازيك,lmnfc1,القصيم
191,ازيك يا احلى,zeenahamdy,None


### Identify most frequent `geo` and `name`

In [33]:
query_cleaned.cleaned_geo.value_counts().head()

None            110
Egypt            11
Mansoura          7
القاهرة, مصر      7
Cairo, Egypt      6
Name: cleaned_geo, dtype: int64

In [34]:
query_cleaned.cleaned_name.value_counts().head()

ashgAltha7ka    3
hramy_kg2       3
nourai208       2
RonzaAskar3     2
cocogeorge83    2
Name: cleaned_name, dtype: int64

In [35]:
query_cleaned.fillna('None')

,cleaned_text,cleaned_name,cleaned_geo
0,ازيك صباح الفل,D38647416,None
2,اللهم آمين .. آمين .. آمين ازيك .. يارب تكون ف...,luminooos,Egypt↩Dubai
3,: اللهم آمين .. آمين .. آمين ازيك .. يارب تكون...,MohammedTalaat1,"القاهرة, مصر"
4,ازيك يا ادهم سليمان —,mohmedelhnawy24,None
5,محمد علي ازيك,s_ss3,بولندا
6,لما البنت اللي بتحبها تكون واقفه مع ابوها وانت...,adoom101,الاسكندرية
7,ازيك يا محمد .. قلقتني عليك بشدة .. اختفائك .. ؟!,Hazem__Azim,Cairo
8,: ازيك يا محمد .. قلقتني عليك بشدة .. اختفائك ...,mzeezm131,None
9,ازيك يا احلي وزه اعمل حسابك تيجيلي كل يوم الاو...,ZZeko1997,None
10,ازيك يامى,PwxRJC67w6wcykP,None


In [36]:
query_cleaned.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 632
Data columns (total 3 columns):
cleaned_text    330 non-null object
cleaned_name    330 non-null object
cleaned_geo     330 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


### Consolidate different geo tags

In [37]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها']
            }

In [38]:
'MAKKAH' in list(locations.values())[0]

True

In [39]:
'MAKKAH' in list(locations.values())[1]

False

In [40]:
locations.values()

dict_values([['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia'], ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait'], ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egy

In [41]:
#pseudo code

for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    elif word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)
    else:
        if word[0] in list(locations.values())[2]:
            query_cleaned['cleaned_geo'].replace(word, 'Egypt', inplace=True)
            

In [42]:
query_cleaned.cleaned_geo

0                           None
2                    Egypt↩Dubai
3                          Egypt
4                           None
5                         بولندا
6                     الاسكندرية
7                          Cairo
8                           None
9                           None
10                          None
12               الأسكندرية, مصر
13                   Tanta Egypt
14             El Monofia, Egypt
15                          None
16                 Cairo , Egypt
17            Ciudad Real, Spain
18                          None
19                          None
20                      Mansoura
22                  Saudi Arabia
23                          None
25                        الدمام
27                       Germany
29                         Egypt
30                          None
31                    اخر الدنيا
32           في ارض الله الواسعه
36              mansoura . egypt
37                          None
38                         Egypt
          

In [43]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
429,: ازيك مصطفي صباحك فل️,moustafakamelm,None
449,: مرة شوية رز حبو يسلمو على مديرهم قالوله ازيك...,rahafbadarneh,المملكة الأردنية الهاشمية
367,: ازيك و أزي المدام و الولاد,sanaasan333,None
472,حتى ازيك مستخسرها مستخسرها مبيقولهاش,KhaledG06308330,mansoura
125,ازيك يا اتش كيفك,wLsRdyB6q14ke8x,None


In [44]:
query_cleaned.to_pickle('../eg_twitter_pickled/stream_ازيك.p')